# Collaboration and Competition

---



### 1. Start the Environment



In [4]:
from unityagents import UnityEnvironment
import numpy as np

env = UnityEnvironment(file_name="./Tennis.app")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [5]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [6]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

Turning environment to inference mode and using a random policy to take actions

In [11]:
for i in range(5):                                         # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(2)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(2, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards             
        #print(rewards)# get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.04500000085681677
Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: 0.04500000085681677
Total score (averaged over agents) this episode: 0.04500000085681677


### 4. Inference Trained Agent using MADPPG

Loading the pretrained model weights for the actors ( as the critic is only required for training not inference), and running a single episode of inference with it. The hyperparameters are only being initialized as the code necessiates them, but they are not being used for inference ( other than the seed init)


In [7]:
import torch

from maddpg import MADDPG

state_size = states.shape[1]
action_size = action_size
num_agents = states.shape[0]
buffer_size = int(1e6)
batch_size = 256 
lr_actor = 3e-4
lr_critic = 3e-4
gamma = 0.99
tau = 1e-3 
weight_decay = 0
noise_min = 0.1
noise_decay= 0.995
OU_mu = 0
OU_theta = 0.15
OU_sigma = 0.05
random_seed = 42


maddpg = MADDPG(state_size= state_size, action_size=action_size,
                buffer_size = buffer_size, batch_size= batch_size,
                num_agents=num_agents, random_seed= random_seed,
               LR_actor=lr_actor, LR_critic= lr_critic, tau = tau, gamma = gamma, 
               noise_min= noise_min, noise_decay=noise_decay, 
               OU_theta = OU_theta, OU_mu=OU_mu, OU_sigma=OU_sigma, weight_decay= weight_decay)

maddpg.load_actors_inference()

In [13]:
for i in range(1):                                         # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                         # initialize the score (for each agent)
    while True:
        actions = maddpg.act(states, 0,train=False)                       # select an action (for each agent)
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards             
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(scores))

Total score (averaged over agents) this episode: [2.60000004 2.60000004]


In [ ]:
# close the environment
env.close()